## James Wilde

#### 6/13/25

#### Homework 7 Part 3

### Part 3

Scrape the list of third party drivers license locations from https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations. but include the link. Save as a CSV.

Tips:

You'll need user-agent headers for this one

In class we always did something like item.find('h2') or the like because there was only ever one h2 we wanted. In this case there are four <td> tags that you want the text from! You'll need to use .find_all and treat them like normal lists ("give me the text from the first one," + "give me the text from the second one" etc etc)

It's okay to have columns you don't want or need! You can always remove them later

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36" 
}

response = requests.get('https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations', headers=headers)
doc = BeautifulSoup(response.text, 'html.parser')

In [2]:
table = doc.find('table')
print(table)

<table class="table table-striped table-hover table-bordered" style="width: 100%;"><thead><tr><td><strong>Company</strong></td>
<td><strong>Address</strong></td>
<td><strong>Telephone</strong></td>
<td><strong>Hours</strong></td>
</tr></thead><tbody><tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
<td>940 N. Alma School Rd., #104<br/>
			Chandler, AZ 85224</td>
<td>480.821.3288</td>
<td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 9:00 a.m.-4:30 p.m.</td>
</tr><tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
<td>5036 W. Cactus Rd., Ste. 2<br/>
			Glendale, AZ 85304</td>
<td>602.264.2400</td>
<td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 8:30 a.m.-4:30 p.m.</td>
</tr><tr><td>Academy of Driving Motor Vehicle Center</td>
<td>4733 E. Broadway Blvd.<br/>
			Tucson, AZ 85711</td>
<td>520.750.7572</td>
<td>Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.</td>
</tr><tr><td>Arizona Auto License</td>
<td>133

In [3]:
rows = table.find_all('tr')
print(rows)

[<tr><td><strong>Company</strong></td>
<td><strong>Address</strong></td>
<td><strong>Telephone</strong></td>
<td><strong>Hours</strong></td>
</tr>, <tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
<td>940 N. Alma School Rd., #104<br/>
			Chandler, AZ 85224</td>
<td>480.821.3288</td>
<td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 9:00 a.m.-4:30 p.m.</td>
</tr>, <tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
<td>5036 W. Cactus Rd., Ste. 2<br/>
			Glendale, AZ 85304</td>
<td>602.264.2400</td>
<td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 8:30 a.m.-4:30 p.m.</td>
</tr>, <tr><td>Academy of Driving Motor Vehicle Center</td>
<td>4733 E. Broadway Blvd.<br/>
			Tucson, AZ 85711</td>
<td>520.750.7572</td>
<td>Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.</td>
</tr>, <tr><td>Arizona Auto License</td>
<td>1337 W. Prince Rd<br/>
			Tucson, AZ 85705</td>
<td>520.696.2023</td>
<td>Driver License Hours: <br

In [4]:
for row in rows[1:]:  # skipping the header row!
    cols = row.find_all('td')

print(cols)

[<td>West Valley Motor Vehicle Title Express LLC</td>, <td>12801 W. Bell Rd. Ste. #113<br/>
			Surprise, AZ 85379</td>, <td>623.977.0929 or fax 623.977.4006</td>, <td>Mon.-Fri. 9 a.m.-5 p.m.</td>]


In [5]:
records = []

for row in rows[1:]:  # skipping the header row!
    cols = row.find_all('td')

    company_cell = cols[0] 
    # ^ storing first cell as a variable 
    
    company_link = company_cell.find('a') 
    # ^ try to find an a tag (which will have url) within the cell
    
    company_name = company_link.text.strip() if company_link else company_cell.get_text(strip=True) 
    # ^ getting company name whether there is or isn't a url in the cell
    
    company_url = company_link['href'] if company_link else ''
    # ^ gets the url if there is one

    address = cols[1].get_text(separator=' ', strip=True)
    telephone = cols[2].get_text(separator=' ', strip=True)
    hours = cols[3].get_text(separator=' ', strip=True)
    # ^ using separator to make sure the outputs have clear spaces
    
    records.append({
        'Company': company_name,
        'Address': address,
        'Telephone': telephone,
        'Hours': hours,
        'Link': company_url
    })

In [7]:
print(records)

[{'Company': '1 Stop Title & Registration Services', 'Address': '940 N. Alma School Rd., #104 Chandler, AZ 85224', 'Telephone': '480.821.3288', 'Hours': 'Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat.\xa09:00 a.m.-4:30 p.m.', 'Link': 'http://az-mvd.com/'}, {'Company': '1 Stop Title & Registration Services', 'Address': '5036 W. Cactus Rd., Ste. 2 Glendale, AZ 85304', 'Telephone': '602.264.2400', 'Hours': 'Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat.\xa08:30 a.m.-4:30 p.m.', 'Link': 'http://az-mvd.com/'}, {'Company': 'Academy of Driving Motor Vehicle Center', 'Address': '4733 E. Broadway Blvd. Tucson, AZ 85711', 'Telephone': '520.750.7572', 'Hours': 'Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.', 'Link': ''}, {'Company': 'Arizona Auto License', 'Address': '1337 W. Prince Rd Tucson, AZ 85705', 'Telephone': '520.696.2023', 'Hours': 'Driver License Hours: Mon.-Fri.\xa09 a.m.-5 p.m. Sat by Appt. Only', 'Link': ''}, {'Company': 'Arizona Auto License Service LLC', 'Address': '1457 N. Eliseo C Felix Jr. Way, Ste. 

In [8]:
df = pd.DataFrame(records)
df.to_csv('az_driver_license_locations.csv', index=False)